In [ ]:
%%capture

%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tfl_training_sbi

In [ ]:
%presentation_style

In [ ]:
%%capture

%set_random_seed 12

In [ ]:
%load_latex_macros

In [ ]:
import math
import matplotlib.pyplot as plt
import sbi.analysis
import sbi.inference
import sbi.utils
import torch
import warnings

from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from tqdm.notebook import trange, tqdm

# from tfl_training_sbi.config import (
#     default_remote_storage,
#     get_config,
#     root_dir,
# )
from tfl_training_sbi.data_utils import (
    SIRSimulation,
    SIRStdScaler,
    load_sir_data,
)
from tfl_training_sbi.utils_sir import eval_sir_model

# set manual seed for reproducibility
_ = torch.manual_seed(0)

# ignore user warnings
warnings.filterwarnings("ignore")

# configure storage location
# storage = default_remote_storage()
# c = get_config(reload=True)

# Neural Likelihood and Ratio Estimation

 - Recap of Neural Posterior Estimation 
 - Introduction to Neural Likelihood Estimation (NLE)
 - Exercise 
 - Neural Ratio Estimation (NRE)
 - Pros & Cons of all three methods
  

## Recap: the Goal of SBI

- We want to learn the posterior $p(\theta | \mathbf{x})$ of a model $\mathcal{M}$ given some data $\mathbf{x}$
- We do so using Bayes rule: 
    $$
    p(\theta | \mathbf{x}) = \frac{p(\mathbf{x} | \theta)
    p(\theta)}{p(\mathbf{x})}
    $$
- However, the Likelihood is usually intractable
- We overcome this challenge by sampling from the joint distribution
  $p(\mathbf{x}, \theta)=p(\mathbf{x}\mid \theta)p(\theta)$ and learn a conditional density estimator


## Recap: Neural Posterior Estimation

- Direct mapping from observations $\mathbf{x}$ to posterior $p(\theta|\mathbf{x})$
- E.g. learning Gaussian mixture, parameterized by neural network

## The Likelihood Function

- The Likelihood function is the probability of the data given the model parameters, i.e. $p(\mathbf{x}|\theta)$
- This is a density function in $\mathbf{x}$ for fixed $\theta$

## Learning the Likelihood Function with a Neural Network

- NLE learns $p(\mathbf{x}|\theta)$ instead of $p(\theta|\mathbf{x})$
- Both are conditional density estimation problems ([NB 02](nb_02_conditional_density_estimation.ipynb))
- NLE is a Maximum Likelihood Estimation problem on $\{(\theta, \mathbf{x})_i
  \}^N_{i=1}$ where $\mathcal{M}(\theta_i) = \mathbf{x}_i$
- $\hat{p}(\theta \mid \mathbf{x}) = p(\theta\mid\mathbf{x})p(\theta)$ is a scaled version with constant $\frac{1}{p(\mathbf{x})}$ 


## Sampling from the NLE Posterior 

MCMC -> bayes training 


## Why should I use NLE

The goal is to obtain the posterior $p(\theta \mid \mathbf{x})$, why learn
$p(\mathbf{x} \mid \theta)$?

 - Reduced complexity due to factorization for i.i.d. observations 
 - Amortized when increasing the hierarchy 
 - Advantage when $\operatorname{dim}(\theta) \gg \operatorname{dim}(\mathbf{x})$
 - Only the Likelihood is intractable 
 - No correction for sequential learning 

## Quick Intro to the `sbi` Toolkit

- Open Source Software for Simulation-Based Inference; <i class="fa-brands fa-github"></i>[GitHub](https://www.mackelab.org/sbi/)
- Provides SNPE, SNLE and SNRE as well as analysis tools out of the box

In [ ]:
# define a uninformative prior
prior = sbi.utils.BoxUniform(
    low=torch.tensor([0.0, 0.0]), high=torch.tensor([1.0, 1.0]) * 2
)


# define a simple simulator
def example_simulator(theta: torch.tensor):
    return torch.sin(theta) + torch.randn_like(theta) * 0.1


# obtain samples from joint distribution
thetas = prior.sample((1_000,))
x = example_simulator(thetas)


# use the first sample as observation
thetas, theta_obs = thetas[1:, :], thetas[0, :]
x, x_obs = x[1:, :], x[0, :]

In [ ]:
# obtain a posterior approx. via NPE
inference = sbi.inference.SNPE(prior=prior, density_estimator="maf")
density_estimator = inference.append_simulations(thetas, x).train()
posterior = inference.build_posterior(density_estimator)

In [ ]:
samples = posterior.sample((100_000,), x=x_obs)
_ = sbi.analysis.pairplot(
    samples=samples,
    points=theta_obs,
    points_colors="r",
    upper="kde",
)

## Example: Application of NLE on the SIR Dataset

We will use the same data as in the previous notebook, but now we will use the
neural likelihood estimator to construct the posterior.

**Task:** Use the provided the pseudo simulator to obtain the SIR dataset. Also,
standardized the data and split it into a training and test set.


In [ ]:
# load the pre-simulated data from disk
data_theta, data_x = load_sir_data("./data/")
simulator = SIRSimulation(
    data_theta[:-1], data_x[:-1]
)

# initialize the scaler and the dataset
# sir_scaler = SIRStdScaler()
# dataset_z = sir_scaler({"theta": torch.tensor(data_theta[:-1]), "obs": torch.tensor(data_x[:-1])})
# theta_z, x_z = dataset_z["theta"], dataset_z["obs"]

# remove the last pair from the dataset
obs_theta, obs_x = torch.tensor(data_theta[-1]).unsqueeze(0), torch.tensor(
    data_x[-1]
).unsqueeze(0)

# data = sir_scaler({"theta": obs_theta, "obs": obs_x})
# obs_theta_z, obs_x_z = data["theta"], data["obs"]

In [ ]:
prior = torch.distributions.LogNormal(
    loc=torch.tensor([math.log(0.4), math.log(0.125)]),
    scale=torch.tensor([0.5, 0.2]),
)
prior = sbi.utils.BoxUniform(low=[0,0], high=[1,1])
prior.sample((10,))

In [ ]:
d, z = data_theta[:1_000, :], data_x[:1_000, :]

In [ ]:
d.shape


In [ ]:
# obtain a posterior approx. via NPE
# todo: the prior is not correct as labels have been scaled
inference = sbi.inference.SNLE(prior=prior, density_estimator="mdn")
density_estimator = inference.append_simulations(torch.tensor(d), torch.tensor(z)).train(
    training_batch_size=128, learning_rate=1e-3, max_num_epochs=5
)
posterior = inference.build_posterior(density_estimator, sample_with="mcmc")

## Let's Investigate the Learned Posterior 

In order to investigate the quality of approximation, sample new $\theta^{\ast}$ from
the prior, evaluate the simulation to obtain $\mathbf{x}^{\ast}$ of such and
sample from the approximation $\hat{p}(\theta \mid \mathbf{x}^{\ast})$ 

In [ ]:
# we will use different pairs from the test set
obs_theta, obs_x = test_theta[0,:], test_x[0,:]

In [ ]:
obs_x.squeeze()

In [ ]:
obs_x.T.shape

In [ ]:
samples = posterior.sample((100,), x=obs_x)
_ = sbi.analysis.pairplot(
    samples=samples,
    # points=obs_theta,
    points_colors="r",
    upper="kde"
)

## Neural Ratio Estimation 

- The Likelihood-ratio is a popular test statistic 
    $$
    r(\mathbf{x} \mid \theta_0, \theta_1) = \frac{p(\mathbf{x} \mid \theta_0)}{p(\mathbf{x} \mid \theta_1)}
    $$
- Cranmer et al (2015) have shown that $r(\mathbf{x}\mid \theta_0, \theta_1)$
  can be expressed by a classifier $d(\mathbf{x}, \theta)$ trained on
  samples from the true $p(\mathbf{x} \mid \theta)$ and an arbitrary but fixed
  hypothisis $p(\mathbf{x} \mid \theta_{\text{ref}})$
    $$
    \begin{align}
    d(\mathbf{x}, \theta) &= p(y=1 \mid \mathbf{x}) =
    \frac{p(\mathbf{x}\mid \theta)}{p(\mathbf{x}\mid \theta) + p(\mathbf{x}\mid
    \theta_{\text{ref}})}  \\
    r(\mathbf{x} \mid \theta_0, \theta_1) &= \frac{d(\mathbf{x},
    \theta)}{ 1 - d(\mathbf{x}, \theta) }
    \end{align}
    $$
  - Thus, learn a classifier to distinguish samples from $p(\mathbf{x},
    \theta)$ from $p(\mathbf{x})p(\theta)$


## Considerations about NRE

- Same concept as NLE but obtaining a normalized posterior 
- Sampling via MCMC 
- Training a classifier is easier than training a density estimator
- $d(\mathbf{x}, \theta)$ will be either $0$ or $1$ for many samples form
  $p(\mathbf{x}, \theta)$, makeing training more difficult

## Usage of NRE

- NRE is also implemented in the `sbi` toolkit

    ```python
    inference = sbi.inference.SNRE(prior)
    density_estimator = inference.append_simulations(thetas, x).train()
    posterior = inference.build_posterior(density_estimator)
    ```

## Practical Considerations for NPE vs. NLE and NRE

- NLE and NRE require MCMC; success depends on the shape and dimension of the
  posterior 
- Sequential approaches are helpful for high dimensional problems or costly
  simulations. However, SNPE might have issues on sharp boundaries of the prior
  and MCMC is slow
- Neural networks can take high dimensional inputs easily. Therefore, use NPE
  when $\dim(\mathbf{x}) \gg \dim(\theta)$ and NLE / NRE when $\dim(\mathbf{x})
  \ll \dim(\theta)$ 
- An appropriate feature extractor should be based on the data's structure. E.g.
  CNNs for images or RNNs for sequential data.

Luckily, SBI and its facets is an active area of research! 


## Summary 

| Method | Pros | Cons |
|:---|:---|:---|
|NPE | Amortized inference | Sequential version requires correction |
|NLE | Likelihood factors for i.i.d. samples, easy usage for hierarchical  problems | Requires MCMC, can be constly in higher dimensions |
|NRE | Likelihood factors for i.i.d. samples, training a classifier is easier than training an NDE | Requires MCMC, possible issues for $d$ close to either $0$ or $1$ |


## References

- sbi: A toolkit for simulation-based inference;
  [GitHub](https://www.mackelab.org/sbi/credits/)
- Cranmer et al. (2015), _"Approximating Likelihood Ratios with Calibrated Discriminative Classifiers"_

<img src="_static/images/aai-institute-cover.svg" alt="Snow" style="width:100%;">
<div class="md-slide title">Thank you for the attention!</div>